In [18]:
import glob
import re

import matplotlib as plt
import numpy as np
import scipy as sp
import xarray as xr

In [2]:
import dask
from dask.distributed import Client, performance_report
from dask_jobqueue import PBSCluster

In [3]:
# File paths
rda_scratch = "/gpfs/csfs1/collections/rda/scratch/harshah"
rda_data = "/gpfs/csfs1/collections/rda/data/"
era5_path = rda_data + "ds633.0/e5.oper.an.sfc/"
zarr_path = rda_scratch + "/tas_zarr/"

In [4]:
## Find NetCDF files with tas (Surface air temperature at 2m) using glob and a search pattern
tas_pattern = era5_path + "**/e5.oper.an.sfc.128_167_2t.*.nc"
tas_ncfiles = glob.glob(tas_pattern, recursive=True)

In [5]:
len(tas_ncfiles)

1009

In [6]:
tas_ncfiles[0]

'/gpfs/csfs1/collections/rda/data/ds633.0/e5.oper.an.sfc/202207/e5.oper.an.sfc.128_167_2t.ll025sc.2022070100_2022073123.nc'

In [7]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name="/dask_job/dask-wk23-hpc",
    cores=1,
    memory="8GiB",
    processes=1,
    local_directory=rda_scratch + "/dask/spill",
    resource_spec="select=1:ncpus=1:mem=8GB",
    queue="casper",
    walltime="5:30:00",
    # interface = 'ib0'
    interface="ext",
)

In [8]:
# GMST function ###
# calculate global means


def get_lat_name(ds):
    for lat_name in ["lat", "latitude"]:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")


def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {"time"}
    return (ds * weight).mean(other_dims)

In [9]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.119:38713,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [10]:
# Define a key function that extracts the YYYYMM part
def extract_date(filename):
    # Using regular expression to find the first occurrence of a pattern resembling 'YYYYMM'
    match = re.search(r"/(\d{6})/", filename)
    if match:
        return match.group(1)
    else:
        return filename  # Return the original filename if pattern is not found


# Sort the list using the key function
sorted_ncfiles = sorted(tas_ncfiles, key=extract_date)
###
sorted_ncfiles[1]

'/gpfs/csfs1/collections/rda/data/ds633.0/e5.oper.an.sfc/194002/e5.oper.an.sfc.128_167_2t.ll025sc.1940020100_1940022923.nc'

In [11]:
#
lat_chunksize = 139
lon_chuksize = 277
time_chunksize = 240

##########


def process_batch(start_index, end_index, ncfile_list, zfile_path):
    # Generate a Zarr store name based on the batch indices
    # ncfile_list = List containing filenames of NetCDF files, # zfile_path = zarr_file_path
    #
    start_date = extract_date(ncfile_list[start_index])
    end_date = extract_date(ncfile_list[end_index - 1])
    #
    zarr_store_name = zfile_path + f"tas2m_{start_date}_{end_date}.zarr"
    print(zarr_store_name)
    # Read the files in the current batch into a single xarray dataset
    datasets = xr.open_mfdataset(
        ncfile_list[start_index:end_index], combine="nested", concat_dim="time"
    ).VAR_2T

    # Rechunk the dataset to holed 10 days-worth of data in a chunk
    rechunked_dataset = datasets.chunk(
        {"time": time_chunksize, "latitude": lat_chunksize, "longitude": lon_chuksize}
    )

    # Save the combined dataset to a Zarr file
    rechunked_dataset.to_zarr(zarr_store_name, mode="w")

## Let us rewrite these files into a zarr store

In [12]:
# Scale the cluster to n workers
cluster.scale(20)

In [19]:
cluster

PBSCluster(9ed13efc, 'tcp://128.117.208.119:38713', workers=20, threads=20, memory=160.00 GiB)

In [15]:
%%time
# Specify batch size
batch_size = 120
for start_index in np.arange(0, len(sorted_ncfiles), batch_size):
    # for start_index in np.arange(0,249,batch_size):
    end_index = min(start_index + batch_size, len(sorted_ncfiles))
    print(end_index)
    # Process the current batch
    process_batch(start_index, end_index, sorted_ncfiles, zarr_path)
    print(f"Processed files {start_index} to {end_index}")

print("All files have been processed.")

1009
/gpfs/csfs1/collections/rda/scratch/harshah/tas_zarr/tas2m_202001_202401.zarr


/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 73.85 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Processed files 960 to 1009
All files have been processed.
CPU times: user 6min 42s, sys: 12.5 s, total: 6min 55s
Wall time: 7min 33s


In [14]:
%%time
tas = xr.open_mfdataset(
    zarr_path + "tas2m*.zarr", combine="nested", concat_dim="time", engine="zarr"
).VAR_2T
tas

CPU times: user 292 ms, sys: 87.6 ms, total: 379 ms
Wall time: 963 ms


<xarray.DataArray 'VAR_2T' (time: 737088, latitude: 721, longitude: 1440)> Size: 3TB
dask.array<concatenate, shape=(737088, 721, 1440), dtype=float32, chunksize=(240, 139, 277), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 6MB 1940-01-01 ... 2024-01-31T23:00:00
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [15]:
%%time
tas_daily = tas.resample(time="1D").mean()
tas_daily

CPU times: user 1min 9s, sys: 1.74 s, total: 1min 11s
Wall time: 1min 17s


<xarray.DataArray 'VAR_2T' (time: 30712, latitude: 721, longitude: 1440)> Size: 128GB
dask.array<stack, shape=(30712, 721, 1440), dtype=float32, chunksize=(1, 139, 277), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 246kB 1940-01-01 1940-01-02 ... 2024-01-31
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [20]:
%%time
tas_daily = tas_daily.chunk({"latitude": 139, "longitude": 544, "time": 1000})
tas_daily

CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.6 ms


<xarray.DataArray 'VAR_2T' (time: 30712, latitude: 721, longitude: 1440)> Size: 128GB
dask.array<rechunk-merge, shape=(30712, 721, 1440), dtype=float32, chunksize=(1000, 139, 544), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 246kB 1940-01-01 1940-01-02 ... 2024-01-31
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

### Let us now compare the writing speed for NetCDF vs zarr formats

In [17]:
%%time
## Generate performance report
with performance_report(filename="e5_zarr_report.html"):
    tas_daily.to_dataset().to_zarr(zarr_path + "e5_tas2m_daily_1940_2023.zarr", mode="w" )

/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 236.53 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 32min 11s, sys: 1min 44s, total: 33min 56s
Wall time: 40min 57s


In [ ]:
%%time
## Generate performance report
with performance_report(filename="e5_nc_report.html"):
    tas_daily.to_dataset().to_netcdf(zarr_path + "e5_tas2m_daily_1940_2023.nc", mode="w")

/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 236.51 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
################################################

### Comapare GMST calculation